<a href="https://colab.research.google.com/github/HakkiAkut/ai-ml-algorithms/blob/master/Perceptron2/Perceptron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing single layer perceptron for classifying mnist digit classification dataset.

Importing datasets

In [ ]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()
#len(train_X)

11501568/11490434 [==============================] - 0s 0us/step


Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

One Hot Encoding

In [ ]:
def one_hot_encoding(data):
  labelEncoder= LabelEncoder() # label encoder
  oneHotEncoder = OneHotEncoder(sparse=False) # one hot encoder
  labels_int = labelEncoder.fit_transform(data)
  labels_int = labels_int.reshape(len(labels_int), 1)
  labels_onehot = oneHotEncoder.fit_transform(labels_int)
  return labels_onehot

Converting 2d list to 1d for easier implementation

In [ ]:
def change2Dto1D(item):
  oneD = np.array(item[0])
  for row in item[1:]:
    oneD = np.concatenate([oneD, row])
  return oneD

data convertion

In [ ]:
def convertingData(data,label):
  new_data= np.array([change2Dto1D(item)for item in data])
  new_label= one_hot_encoding(label)
  return new_data, new_label

In [ ]:
train_X1D,train_y1H= convertingData(train_X[0:20000],train_y[0:20000])
test_X1D, test_y1H= convertingData(test_X[0:7000],test_y[0:7000])
#train_X1D,train_y1H[0]

Multi-class Perceptron Training function

In [ ]:
def multiclass_training(features, labels, lr=0.001, iteration=5):
  weights=[]
  bias=[]
  for j in range(len(labels[0])):
    weights.append(np.random.rand(features.shape[1])) 
    bias.append(0.0) 

  # training the same dataset for 'iteration' times
  for i in range(iteration):
    index = 0;
    #training each row
    for sample in features:
      pred = multiclass_prediction(sample, weights, bias)
      # updating weights and bias for each column
      for k in range(len(bias)):
        delta_weights= lr*(labels[index][k]- pred[k])
        weights[k]+= delta_weights*sample
        bias[k] += delta_weights
      index+=1
  return weights,bias

Multi-class Prediction Function

In [ ]:
def multiclass_prediction(features, weights, bias):
  calculated_inputs=[]
  # predicting for each rows of one hot encoded labels value, 
  for k in range(len(bias)):
    calc_inp = np.dot(features, weights[k])+bias[k]
    calculated_inputs.append(calc_inp)
  gp=0; 
  # finding greatest probability and changing to one hot encoded type.
  for j in range(len(calculated_inputs)):
    if calculated_inputs[j] > calculated_inputs[gp]:
      gp= j
  pred=np.zeros(len(calculated_inputs))
  pred[gp]= 1
  return pred 

In [ ]:
def check_prediction_list(prediction, result):
  checked = prediction == result 
  if type(checked)== bool:
    print(checked,prediction,result)
    if checked == False:
      return 1
    else:
      return 0
  else:
    if False in checked: 
      return 1
    else:
      return 0

In [ ]:
def find_metrics(labels,check_list,decision_list):
  n = len(check_list)
  error=0
  label_num= np.zeros(len(labels[0])) # number of real values
  label_true= np.zeros(len(labels[0])) # number of true predicts
  label_pred= np.zeros(len(labels[0])) # number of predicted values 
  precision_list= np.zeros(len(labels[0]))
  recall_list= np.zeros(len(labels[0]))
  # calculating number of real values, true predicts, predicted values 
  for k in range(n):
    index = find_index(labels[k])
    label_num[index] = label_num[index] + 1
    if check_list[k] == 0:
      label_true[index] = label_true[index] + 1
      label_pred[index] = label_pred[index] + 1
    else:
      index_choosed = find_index(decision_list[k])
      label_pred[index_choosed] = label_pred[index_choosed] + 1
      error +=1
  # calculating each precision and recall  
  for j in range(len(label_num)):
    recall_list[j]= label_true[j]/label_num[j]
    precision_list[j]= label_true[j]/label_pred[j]
  # calculating average precision and recall 
  recall_avg= sum(recall_list)/len(recall_list)
  precision_avg= sum(precision_list)/len(precision_list)
  # printing the metrics
  print(f"average accuracy {(n-error)/n}") 
  print(f"average recall {recall_avg}")
  print(f"average precision {precision_avg}")  

def find_index(label):
  index=0
  for k in range(len(label)):
    if label[k] == 1:
      return k 

In [ ]:
weights_trained, bias_trained = multiclass_training(train_X1D, train_y1H, lr=0.01, iteration=50)
check_list= []
predictions= []
error_num= 0

zipper = zip(test_X1D, test_y1H)
for sample , result in zipper:
  prediction_final = multiclass_prediction(sample, weights_trained, bias_trained)
  predictions.append(prediction_final)
  error = check_prediction_list(prediction_final, result)
  check_list.append(error)
  error_num = error_num + error

find_metrics(train_y1H,check_list,predictions)

average accuracy 0.8751428571428571
average recall 0.8751727914552431
average precision 0.877105360279655


average accuracy 0.8751428571428571
average recall 0.8751727914552431
average precision 0.877105360279655